In [ ]:
!pip install pytrends

In [ ]:
!pip install seaborn

In [4]:
from pytrends.request import TrendReq
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import json
import time

In [5]:
timezone_offset = -330 # INDIA
# timezone_offset = -480 # Phillipines

pytrends = TrendReq(hl='en-US', tz=timezone_offset, retries=10, backoff_factor=0.5)

In [6]:
date_entry = input('Enter a start date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

Enter a start date in YYYY-MM-DD format: 2020-01-01


In [7]:
date_entry = input('Enter a end date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date2 = datetime.date(year, month, day)

Enter a end date in YYYY-MM-DD format: 2020-03-31


In [8]:
total_t_frame = date1.strftime("%Y-%m-%d") + " " + date2.strftime("%Y-%m-%d")
total_t_frame

'2020-01-01 2020-03-31'

In [9]:
enddates = pd.date_range(date1, date2 , freq='1M').tolist()
startdates = pd.to_datetime([x.date() - pd.tseries.offsets.MonthBegin(1) for x in enddates]).tolist()

In [10]:
t_frames = []
for i in range(len(startdates)):
    s = startdates[i].date().strftime("%Y-%m-%d")
    e = enddates[i].date().strftime("%Y-%m-%d")
    t_frames.append(s + " " + e)

In [11]:
t_frames

['2020-01-01 2020-01-31', '2020-02-01 2020-02-29', '2020-03-01 2020-03-31']

In [12]:
cat_df = pd.read_csv("categories_defined.csv")
cat_names = cat_df["name"].tolist()
cat_ids = cat_df["id"].tolist()

In [14]:
from pandas.io.json._normalize import nested_to_record

def get_related_topics(catno, timeframe):
    """
    Gets related queries
    """
    kw_list = [""]
    # Create Pytrend Client
    pytrends.build_payload(cat=catno, kw_list=kw_list, timeframe=timeframe, geo="PH")
    
    # Form Request
    related_payload = dict()
    request_json = pytrends.related_topics_widget_list[0]
    related_payload["req"] = json.dumps(request_json["request"])
    related_payload["token"] = request_json["token"]
    related_payload["tz"] = pytrends.tz

    # Send Request
    req_json = pytrends._get_data(
        url=TrendReq.RELATED_QUERIES_URL,
        method=TrendReq.GET_METHOD,
        trim_chars=5,
        params=related_payload,
    )
    
    # return req_json
    # Tabulate Rising & Top searches.
    try:
        temp = req_json["default"]["rankedList"][0]["rankedKeyword"]
        top_df = pd.DataFrame([nested_to_record(d, sep='_') for d in temp])[["topic_title", "topic_type", "formattedValue"]]
    except KeyError:
        # in case no top queries are found, the lines above will throw a KeyError
        top_df = pd.DataFrame(columns=["topic_title", "topic_type", "formattedValue"])

    # rising queries
    try:
        temp = req_json["default"]["rankedList"][1]["rankedKeyword"]
        rising_df = pd.DataFrame([nested_to_record(d, sep='_') for d in temp])[["topic_title", "topic_type", "formattedValue"]]
    except KeyError:
        # in case no rising queries are found, the lines above will throw a KeyError
        rising_df = pd.DataFrame(columns=["topic_title", "topic_type", "formattedValue"])

    return(top_df,rising_df)

In [15]:
datetime.datetime.strptime(t_frames[0].split()[0], "%Y-%m-%d").strftime("%b-%Y")

'Jan-2020'

In [16]:
related_topics_monthly = pd.DataFrame()
for i in range(len(cat_ids)):
    print(i, cat_ids[i], cat_names[i])
    rt_top_df, rt_rising_df = get_related_topics(cat_ids[i], total_t_frame)
    
    if(len(rt_top_df)==0 or len(rt_top_df)==0):
        print("skipping ", cat_names[i], cat_ids[i])
        continue
        
    for j in range(len(t_frames)):
        if(i%20==0):
            time.sleep(20)

        rt_top_df, rt_rising_df = get_related_topics(cat_ids[i], timeframe=t_frames[j])
        
        if(len(rt_top_df)==0):
            print(cat_names[i], "has no top topics in", t_frames[j])
        if(len(rt_rising_df)==0):
            print(cat_names[i], "has no rising topics in", t_frames[j])

        rt_top_df.columns = ["topic_title", "topic_type", "subject"]
        rt_rising_df.columns = ["topic_title", "topic_type", "subject"]

        rt_top_df["related_topics"] = "top"
        rt_rising_df["related_topics"] = "rising"
        rt_df = rt_rising_df.append(rt_top_df)

        rt_df = rt_top_df.append(rt_rising_df)

        rt_df["geo"] = "PH"
        rt_df["keyword"] = cat_names[i]
        rt_df["category"] = cat_ids[i]
        rt_df["timeframe"] = datetime.datetime.strptime(t_frames[j].split()[0], "%Y-%m-%d").strftime("%b-%Y")
        related_topics_monthly = related_topics_monthly.append(rt_df)

0 277 Alcoholic Beverages


ResponseError: The request failed: Google returned a response with code 429.

In [15]:
related_topics_monthly.reset_index()

,index,topic_title,topic_type,subject,related_topics,geo,keyword,category,timeframe
0,0,Alcoholic drink,Topic,100,top,PH,Alcoholic Beverages,277,Jan-2020
1,1,Meralco Bolts,Basketball team,99,top,PH,Alcoholic Beverages,277,Jan-2020
2,2,Beer,Alcoholic drink,98,top,PH,Alcoholic Beverages,277,Jan-2020
3,3,Barangay Ginebra San Miguel,Basketball team,97,top,PH,Alcoholic Beverages,277,Jan-2020
4,4,Wine,Alcoholic drink,73,top,PH,Alcoholic Beverages,277,Jan-2020
...,...,...,...,...,...,...,...,...,...
3034,20,Zipper,Topic,Breakout,rising,PH,Casual Apparel,984,Feb-2020
3035,21,Mickey Mouse,Cartoon character,Breakout,rising,PH,Casual Apparel,984,Feb-2020
3036,22,FUCT,Clothing,Breakout,rising,PH,Casual Apparel,984,Feb-2020
3037,23,Samsung,Topic,Breakout,rising,PH,Casual Apparel,984,Feb-2020


In [16]:
related_topics_monthly.reset_index().to_csv("related_topics_monthly.csv")

In [3]:
import datetime
datetime.datetime.now().strftime("%Y-%m-%d")

'2020-07-23'